In [2]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
from sklearn import linear_model
import numpy as np
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error

from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler


In [3]:
def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')

In [4]:
from implicit import bpr
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
from surprise import Prediction

In [5]:
# Load gzipped CSV
df = pd.read_csv("train_Interactions.csv.gz", header=0)

# Optional: inspect
print(df.head())
print(df.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'train_Interactions.csv.gz'

In [ ]:
allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)

for user,book,r in readCSV("train_Interactions.csv.gz"):
  r = int(r)
  allRatings.append(r)
  userRatings[user].append(r)
  itemRatings[book].append(r)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
itemAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
for i in itemRatings:
  itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

In [ ]:
# Prepare for surprise
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['userID', 'bookID', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
def train_with_early_stop2(trainset, testset,
                          n_factors=1, lr_all=0.005,
                          reg_bias = 0.05, reg_factor = 0.05, 
                          max_epochs=50, patience=3, verbose=True):
    
    
    best_mse = np.inf
    best_epoch = 0
    no_improve = 0
    history = []

    best_params = {}

    for epoch in range(1, max_epochs + 1):
        model = SVD(n_factors=n_factors, reg_bu=reg_bias, reg_bi=reg_bias, reg_pu=reg_factor, reg_qi=reg_factor, lr_all=lr_all, n_epochs=epoch)
        model.fit(trainset)
        preds = model.test(testset)
        clipped_preds = [p._replace(est=min(max(p.est, 0), 5)) for p in preds]


        mse = accuracy.mse(clipped_preds, verbose=False)
        history.append((epoch, mse))

        if verbose:
            print(f"Epoch {epoch:2d} | MSE = {mse:.6f}")

        if mse + 1e-6 < best_mse:
            best_mse = mse
            best_epoch = epoch
            best_params = {
                "bu": model.bu.copy(),
                "bi": model.bi.copy(),
                "pu": model.pu.copy(),
                "qi": model.qi.copy()
            }
            no_improve = 0
        else:
            no_improve += 1

        if no_improve >= patience:
            if verbose:
                print(f"Early stopping at epoch {best_epoch} (best MSE = {best_mse:.6f})\n")
            break

    # restore best weights
    model.bu = best_params["bu"]
    model.bi = best_params["bi"]
    model.pu = best_params["pu"]
    model.qi = best_params["qi"]

    return model, history, best_mse, best_epoch

In [ ]:
reg_biases = [0.19, 0.2, 1, 3, 5, 10]
reg_factors = [0.5]
lrs = [0.005]

results = []
for regbias in reg_biases:
    for regfactor in reg_factors:
        for lr in lrs:
            print(f"\nTraining with reg_bias={regbias}, reg_factor={regfactor}, learning_rate={lr}")
            _, _, best_mse, best_epoch = train_with_early_stop2(
                trainset, testset,
                n_factors=1,
                reg_bias = regbias, reg_factor = regfactor, 
                lr_all=lr,
                max_epochs=60,
                patience=5,
                verbose=False
            )
            results.append((regbias, regfactor, lr, best_mse, best_epoch))

# sort by MSE
results.sort(key=lambda x: x[3])
print("\nTop results:")
for regbias, regfactor, lr, mse, ep in results[:5]:
    print(f"reg_bias={regbias:<6} reg_factor={regfactor:<6} learning_rate={lr:<6} best_mse={mse:.6f}  epoch={ep}")


Training with reg_bias=0.19, reg_factor=0.5, learning_rate=0.005


NameError: name 'train_with_early_stop2' is not defined

In [ ]:
reg_biases = [0.19, 0.2]
reg_factors = [0.57, 0.59, 0.6, 0.61, 0.63]
lrs = [0.005]

results = []
for regbias in reg_biases:
    for regfactor in reg_factors:
        for lr in lrs:
            print(f"\nTraining with reg_bias={regbias}, reg_factor={regfactor}, learning_rate={lr}")
            _, _, best_mse, best_epoch = train_with_early_stop2(
                trainset, testset,
                n_factors=2,
                reg_bias = regbias, reg_factor = regfactor, 
                lr_all=lr,
                max_epochs=60,
                patience=5,
                verbose=False
            )
            results.append((regbias, regfactor, lr, best_mse, best_epoch))

# sort by MSE
results.sort(key=lambda x: x[3])
print("\nTop results:")
for regbias, regfactor, lr, mse, ep in results[:5]:
    print(f"reg_bias={regbias:<6} reg_factor={regfactor:<6} learning_rate={lr:<6} best_mse={mse:.6f}  epoch={ep}")

In [ ]:
reg_biases = [0.17, 0.175, 0.18]
reg_factors = [0.165, 0.17, 0.175]

lrs = [0.005]

results = []
for regbias in reg_biases:
    for regfactor in reg_factors:
        for lr in lrs:
            print(f"\nTraining with reg_bias={regbias}, reg_factor={regfactor}, learning_rate={lr}")
            _, _, best_mse, best_epoch = train_with_early_stop2(
                trainset, testset,
                n_factors=3,
                reg_bias = regbias, reg_factor = regfactor, 
                lr_all=lr,
                max_epochs=60,
                patience=5,
                verbose=False
            )
            results.append((regbias, regfactor, lr, best_mse, best_epoch))

# sort by MSE
results.sort(key=lambda x: x[3])
print("\nTop results:")
for regbias, regfactor, lr, mse, ep in results[:5]:
    print(f"reg_bias={regbias:<6} reg_factor={regfactor:<6} learning_rate={lr:<6} best_mse={mse:.6f}  epoch={ep}")

In [ ]:
model = SVD(n_factors=1, reg_bu=0.2, reg_bi=0.2, reg_pu=0.5, reg_qi=0.5, lr_all=0.005, n_epochs=38, random_state=42)
model.fit(trainset)
preds = model.test(testset)
clipped_preds = [p._replace(est=min(max(p.est, 0), 5)) for p in preds]
mse = accuracy.mse(clipped_preds, verbose=False)
print("MSE:", mse)

MSE: 1.4557479258776624


In [20]:
full_trainset = data.build_full_trainset()

model = SVD(
    n_factors=1,
    reg_bu=0.2,
    reg_bi=0.2,
    reg_pu=0.5,
    reg_qi=0.5,
    lr_all=0.005,
    n_epochs=38,
    random_state=42
)
model.fit(full_trainset)


In [23]:
with open("predictions_Rating.csv", 'w') as pred_file:
    for l in open("pairs_Rating.csv"):
        if l.startswith("userID"):
            pred_file.write(l)
            continue
            
        u, b = l.strip().split(',')
        
        user_seen = u in userAverage
        book_seen = b in itemAverage
        

        # Case 1: both unseen
        if not user_seen and not book_seen:
            r_hat = globalAverage
        
        # Case 2: user unseen, book seen
        elif not user_seen:
            r_hat = itemAverage[b]
        
        # Case 3: book unseen, user seen
        elif not book_seen:
            r_hat = userAverage[u]

        # Case 4: both seen → use SVD model
        else:
            pred = model.predict(u, b)
            r_hat = pred.est
        
        # Clip to [0,5]
        r_hat = min(max(r_hat, 0), 5)

        pred_file.write(f"{u},{b},{r_hat}\n")
